# End-to-End Multi-Model Pipeline — Train / Validate / Test
## 70 / 15 / 15 Split — Final Evaluation on Unseen Test Set

**Pipeline**:  
```
Raw CIR → Stage 1 (PI-HLNN: LOS/NLOS?) → Stage 2 (MLP: Single/Multi bounce?) → Stage 3 (MLP: Predict bias) → d_corrected
```

**Split**: 70% Train / 15% Validation / 15% Test (matching Xu Xueli 2024)  
**Test set**: Completely unseen — never used during training or hyperparameter tuning  

| Stage | Model | Input | Output |
|-------|-------|-------|--------|
| 1 | PI-HLNN (Liquid Neural Network) | Raw 60-sample CIR window | LOS (0) / NLOS (1) |
| 2 | MLP Classifier (32→16→1) | 6 CIR physics features | Single-bounce (0) / Multi-bounce (1) |
| 3 | MLP Regressor (64→32→1) | 6 CIR physics features | NLOS bias (meters) |

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns
import re
import copy
from scipy.signal import find_peaks
from scipy.stats import kurtosis as scipy_kurtosis
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay,
    classification_report, r2_score,
    mean_absolute_error, mean_squared_error
)
from torch.utils.data import DataLoader, TensorDataset

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

Device: cuda


---
## Section 1: Data Loading & 70/15/15 Split

Stratified split ensuring LOS/NLOS ratio preserved in all 3 sets.

In [2]:
# ==========================================
# GROUND TRUTH
# ==========================================
GROUND_TRUTH = {
    '7.79m':  {'d_direct': 7.79,  'd_bounce': 12.79, 'bias': 5.00},
    '10.77m': {'d_direct': 10.77, 'd_bounce': 16.09, 'bias': 5.32},
    '14m':    {'d_direct': 14.00, 'd_bounce': 16.80, 'bias': 2.80},
}

def get_distance_group(fname):
    match = re.match(r'^([\d.]+m)', str(fname))
    return match.group(1) if match else None

# ==========================================
# LOAD FULL DATASET
# ==========================================
df = pd.read_csv('../dataset/channels/combined_uwb_dataset.csv')
print(f'Total samples: {len(df)}')
print(f'  LOS:  {(df["Label"]==0).sum()}')
print(f'  NLOS: {(df["Label"]==1).sum()}')

# ==========================================
# 70 / 15 / 15 STRATIFIED SPLIT
# ==========================================
# First split: 70% train, 30% temp
train_df, temp_df = train_test_split(
    df, test_size=0.30, random_state=SEED, stratify=df['Label']
)
# Second split: 50/50 of temp -> 15% val, 15% test
val_df, test_df = train_test_split(
    temp_df, test_size=0.50, random_state=SEED, stratify=temp_df['Label']
)

train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

print(f'\nSplit Results:')
print(f'  Train: {len(train_df)} ({100*len(train_df)/len(df):.1f}%) — LOS={int((train_df["Label"]==0).sum())}, NLOS={int((train_df["Label"]==1).sum())}')
print(f'  Val:   {len(val_df)} ({100*len(val_df)/len(df):.1f}%) — LOS={int((val_df["Label"]==0).sum())}, NLOS={int((val_df["Label"]==1).sum())}')
print(f'  Test:  {len(test_df)} ({100*len(test_df)/len(df):.1f}%) — LOS={int((test_df["Label"]==0).sum())}, NLOS={int((test_df["Label"]==1).sum())}')

Total samples: 3600
  LOS:  1800
  NLOS: 1800

Split Results:
  Train: 2520 (70.0%) — LOS=1260, NLOS=1260
  Val:   540 (15.0%) — LOS=270, NLOS=270
  Test:  540 (15.0%) — LOS=270, NLOS=270


---
## Section 2: Shared Preprocessing & Feature Extraction

Functions shared by all 3 stages.

In [3]:
CIR_COLS = sorted(
    [c for c in df.columns if c.startswith('CIR')],
    key=lambda x: int(x.replace('CIR', ''))
)

# ==========================================
# ROI ALIGNMENT (shared)
# ==========================================
def get_roi_alignment(sig, search_start=700, search_end=800):
    region = sig[search_start:search_end]
    if len(region) == 0:
        return np.argmax(sig)
    peak_local = np.argmax(region)
    peak_idx = search_start + peak_local
    peak_val = sig[peak_idx]
    noise_section = sig[:search_start]
    if len(noise_section) > 10:
        noise_mean = np.mean(noise_section)
        noise_std = np.std(noise_section)
        threshold = max(noise_mean + 3 * noise_std, 0.05 * peak_val)
    else:
        threshold = 0.05 * peak_val
    leading_edge = peak_idx
    for i in range(peak_idx, max(search_start - 20, 0), -1):
        if sig[i] < threshold:
            leading_edge = i + 1
            break
    return leading_edge


# ==========================================
# STAGE 1 PREPROCESSING: Raw CIR -> 60-sample window
# ==========================================
S1_CONFIG = {
    'pre_crop': 10, 'post_crop': 50, 'total_len': 60,
    'hidden_size': 64, 'input_size': 1, 'dropout': 0.35,
    'batch_size': 32, 'max_epochs': 50, 'lr': 3e-3,
    'weight_decay': 1e-4, 'patience': 12,
    'scheduler_patience': 7, 'scheduler_factor': 0.5,
    'grad_clip': 1.0,
}

def preprocess_stage1(data_df):
    """Convert DataFrame to Stage 1 input: (N, 60, 1) CIR windows."""
    PRE, TOTAL = S1_CONFIG['pre_crop'], S1_CONFIG['total_len']
    seqs, labels = [], []
    for _, row in data_df.iterrows():
        sig = pd.to_numeric(row[CIR_COLS], errors='coerce').fillna(0).astype(float).values
        rxpacc = float(row.get('RXPACC', row.get('RX_PACC', 128.0)))
        if rxpacc > 0:
            sig = sig / rxpacc
        le = get_roi_alignment(sig)
        start = max(0, le - PRE)
        end = start + TOTAL
        if end > len(sig):
            end = len(sig)
            start = max(0, end - TOTAL)
        crop = sig[start:end]
        if len(crop) < TOTAL:
            crop = np.pad(crop, (0, TOTAL - len(crop)), mode='constant')
        local_min, local_max = np.min(crop), np.max(crop)
        rng = local_max - local_min
        crop = (crop - local_min) / rng if rng > 0 else np.zeros(TOTAL)
        seqs.append(crop)
        labels.append(float(row['Label']))
    X = np.array(seqs).reshape(-1, TOTAL, 1).astype(np.float32)
    y = np.array(labels).astype(np.float32)
    return X, y


# ==========================================
# STAGES 2 & 3 FEATURE EXTRACTION: CIR -> 6 physics features
# ==========================================
FEATURE_NAMES = [
    'Kurtosis', 'Rise_Time', 'RMS_Delay_Spread',
    'Mean_Excess_Delay', 'Power_Ratio', 'Num_Peaks'
]

PEAK_CONFIG = {
    'peak_prominence': 0.20, 'peak_min_distance': 5,
    'single_bounce_max_peaks': 2,
    'search_start': 700, 'search_end': 800,
}

def extract_cir_features(sig, leading_edge):
    ss, se = PEAK_CONFIG['search_start'], PEAK_CONFIG['search_end']
    peak_idx = np.argmax(sig[ss:se]) + ss
    win_start = max(0, peak_idx - 10)
    win_end = min(len(sig), peak_idx + 30)
    window = sig[win_start:win_end]
    kurt = scipy_kurtosis(window, fisher=True) if len(window) > 4 else 0.0
    rise_time = float(peak_idx - leading_edge)
    pdp_start = max(0, leading_edge)
    pdp_sig = sig[pdp_start:min(pdp_start + 150, len(sig))]
    pdp = pdp_sig ** 2
    total_pdp = np.sum(pdp)
    if total_pdp > 0:
        times = np.arange(len(pdp), dtype=float)
        mean_delay = np.sum(pdp * times) / total_pdp
        second_moment = np.sum(pdp * times ** 2) / total_pdp
        rms_spread = np.sqrt(max(0, second_moment - mean_delay ** 2))
    else:
        mean_delay, rms_spread = 0.0, 0.0
    fp_start = max(0, leading_edge - 1)
    fp_end = min(len(sig), leading_edge + 2)
    fp_energy = np.sum(sig[fp_start:fp_end] ** 2)
    total_energy = np.sum(sig ** 2)
    power_ratio = fp_energy / total_energy if total_energy > 0 else 0.0
    roi_start = max(0, leading_edge - 5)
    roi_end = min(len(sig), leading_edge + 120)
    roi = sig[roi_start:roi_end]
    if len(roi) > 0 and np.max(roi) > 0:
        roi_norm = roi / np.max(roi)
        peaks, _ = find_peaks(roi_norm, prominence=PEAK_CONFIG['peak_prominence'],
                              distance=PEAK_CONFIG['peak_min_distance'])
        num_peaks = len(peaks)
    else:
        num_peaks = 0
    return {
        'Kurtosis': kurt, 'Rise_Time': rise_time,
        'RMS_Delay_Spread': rms_spread, 'Mean_Excess_Delay': mean_delay,
        'Power_Ratio': power_ratio, 'Num_Peaks': float(num_peaks),
    }


def extract_features_from_df(data_df):
    """Extract 6 CIR features + metadata from DataFrame."""
    features_list, source_files, distances = [], [], []
    for _, row in data_df.iterrows():
        sig = pd.to_numeric(row[CIR_COLS], errors='coerce').fillna(0).astype(float).values
        rxpacc = float(row.get('RXPACC', row.get('RX_PACC', 128.0)))
        if rxpacc > 0:
            sig = sig / rxpacc
        le = get_roi_alignment(sig)
        feats = extract_cir_features(sig, le)
        features_list.append(feats)
        source_files.append(str(row['Source_File']))
        distances.append(float(row['Distance']))
    return pd.DataFrame(features_list), source_files, distances


print('Preprocessing functions defined.')
print(f'CIR columns: {len(CIR_COLS)}')

Preprocessing functions defined.
CIR columns: 1016


---
## Section 3: Model Architectures

All 3 model classes — identical to the individual stage notebooks.

In [4]:
# ==========================================
# STAGE 1: PI-HLNN (Liquid Neural Network)
# ==========================================
class PILiquidCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.synapse = nn.Linear(input_size + hidden_size, hidden_size)
        self.tau_net = nn.Sequential(
            nn.Linear(input_size + hidden_size, 32),
            nn.Tanh(),
            nn.Linear(32, hidden_size),
            nn.Sigmoid()
        )
        self.A = nn.Parameter(torch.ones(hidden_size) * -0.5)

    def forward(self, x_t, h_prev, dt=1.0):
        combined = torch.cat([x_t, h_prev], dim=1)
        tau = 1.0 + 5.0 * self.tau_net(combined)
        S_t = torch.tanh(self.synapse(combined))
        numerator = h_prev + (dt * S_t * self.A)
        denominator = 1.0 + (dt / tau)
        h_new = numerator / denominator
        return h_new, tau


class PI_HLNN(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, dropout=0.35):
        super().__init__()
        self.hidden_size = hidden_size
        self.cell = PILiquidCell(input_size, hidden_size)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 32),
            nn.SiLU(),
            nn.Dropout(dropout),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x_seq, return_dynamics=False):
        batch_size, seq_len, _ = x_seq.size()
        h_t = torch.zeros(batch_size, self.hidden_size, device=x_seq.device)
        h_sum = torch.zeros_like(h_t)
        tau_sum = torch.zeros_like(h_t)
        for t in range(seq_len):
            h_t, tau_t = self.cell(x_seq[:, t, :], h_t)
            h_sum += h_t
            tau_sum += tau_t
        h_pooled = h_sum / seq_len
        tau_mean = tau_sum / seq_len
        prediction = self.classifier(h_pooled)
        return prediction, tau_mean


# ==========================================
# STAGE 2: Bounce Classifier (MLP)
# ==========================================
class BounceClassifier(nn.Module):
    def __init__(self, input_size=6, hidden_layers=[32, 16], dropout=0.3):
        super().__init__()
        layers = []
        prev_size = input_size
        for h_size in hidden_layers:
            layers.extend([nn.Linear(prev_size, h_size), nn.ReLU(), nn.Dropout(dropout)])
            prev_size = h_size
        layers.append(nn.Linear(prev_size, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)


# ==========================================
# STAGE 3: NLOS Bias Predictor (MLP)
# ==========================================
class NLOSBiasPredictor(nn.Module):
    def __init__(self, input_size=6, hidden_layers=[64, 32], dropout=0.2):
        super().__init__()
        layers = []
        prev_size = input_size
        for h_size in hidden_layers:
            layers.extend([nn.Linear(prev_size, h_size), nn.ReLU(), nn.Dropout(dropout)])
            prev_size = h_size
        layers.append(nn.Linear(prev_size, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)


print('All 3 model architectures defined.')
print(f'  Stage 1 (PI-HLNN):         {sum(p.numel() for p in PI_HLNN().parameters()):,} params')
print(f'  Stage 2 (BounceClassifier): {sum(p.numel() for p in BounceClassifier().parameters()):,} params')
print(f'  Stage 3 (NLOSBiasPredictor):{sum(p.numel() for p in NLOSBiasPredictor().parameters()):,} params')

All 3 model architectures defined.
  Stage 1 (PI-HLNN):         10,625 params
  Stage 2 (BounceClassifier): 769 params
  Stage 3 (NLOSBiasPredictor):2,561 params


---
## Section 4: Train All 3 Stages on 70% Train Set

Each stage trained independently. Validation set (15%) used for early stopping.

In [5]:
# ==========================================
# STAGE 1 TRAINING: PI-HLNN (LOS/NLOS)
# ==========================================
print('STAGE 1: Training PI-HLNN on 70% train set...')
print('='*60)

# Preprocess
X_train_s1, y_train_s1 = preprocess_stage1(train_df)
X_val_s1, y_val_s1 = preprocess_stage1(val_df)
print(f'  Train: {X_train_s1.shape}, Val: {X_val_s1.shape}')

# To tensors
X_tr1 = torch.tensor(X_train_s1).to(device)
y_tr1 = torch.tensor(y_train_s1).unsqueeze(1).to(device)
X_va1 = torch.tensor(X_val_s1).to(device)
y_va1 = torch.tensor(y_val_s1).unsqueeze(1).to(device)

train_ds1 = TensorDataset(X_tr1, y_tr1)
train_loader1 = DataLoader(train_ds1, batch_size=S1_CONFIG['batch_size'], shuffle=True)

# Model
model_s1 = PI_HLNN(
    input_size=S1_CONFIG['input_size'],
    hidden_size=S1_CONFIG['hidden_size'],
    dropout=S1_CONFIG['dropout']
).to(device)

criterion_s1 = nn.BCELoss()
optimizer_s1 = optim.AdamW(model_s1.parameters(), lr=S1_CONFIG['lr'],
                           weight_decay=S1_CONFIG['weight_decay'])
scheduler_s1 = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer_s1, mode='max', factor=S1_CONFIG['scheduler_factor'],
    patience=S1_CONFIG['scheduler_patience'])

best_val_acc_s1 = 0
best_state_s1 = None
patience_s1 = 0

for epoch in range(S1_CONFIG['max_epochs']):
    model_s1.train()
    for bx, by in train_loader1:
        optimizer_s1.zero_grad()
        pred, tau = model_s1(bx)
        loss = criterion_s1(pred, by)
        loss.backward()
        nn.utils.clip_grad_norm_(model_s1.parameters(), S1_CONFIG['grad_clip'])
        optimizer_s1.step()

    model_s1.eval()
    with torch.no_grad():
        vp, _ = model_s1(X_va1)
        val_acc = ((vp > 0.5).float() == y_va1).float().mean().item()
    scheduler_s1.step(val_acc)

    if val_acc > best_val_acc_s1:
        best_val_acc_s1 = val_acc
        best_state_s1 = copy.deepcopy(model_s1.state_dict())
        patience_s1 = 0
    else:
        patience_s1 += 1

    if epoch % 5 == 0:
        print(f'  Ep {epoch:>3} | Val Acc: {100*val_acc:.2f}% | Best: {100*best_val_acc_s1:.2f}%')
    if patience_s1 >= S1_CONFIG['patience']:
        print(f'  Early stopping at epoch {epoch}')
        break

model_s1.load_state_dict(best_state_s1)
print(f'\nStage 1 Best Val Accuracy: {100*best_val_acc_s1:.2f}%')

STAGE 1: Training PI-HLNN on 70% train set...
  Train: (2520, 60, 1), Val: (540, 60, 1)
  Ep   0 | Val Acc: 75.19% | Best: 75.19%
  Ep   5 | Val Acc: 97.41% | Best: 97.41%
  Ep  10 | Val Acc: 97.41% | Best: 97.96%
  Ep  15 | Val Acc: 99.63% | Best: 99.63%
  Ep  20 | Val Acc: 98.15% | Best: 99.81%
  Ep  25 | Val Acc: 96.11% | Best: 99.81%
  Ep  30 | Val Acc: 99.81% | Best: 100.00%
  Ep  35 | Val Acc: 99.63% | Best: 100.00%
  Ep  40 | Val Acc: 99.81% | Best: 100.00%
  Early stopping at epoch 40

Stage 1 Best Val Accuracy: 100.00%


In [6]:
# ==========================================
# STAGE 2 TRAINING: Bounce Classifier (NLOS only)
# ==========================================
print('STAGE 2: Training Bounce Classifier on NLOS subset of train set...')
print('='*60)

S2_CONFIG = {
    'hidden_layers': [32, 16], 'dropout': 0.3,
    'batch_size': 32, 'max_epochs': 100, 'lr': 1e-3,
    'weight_decay': 1e-4, 'patience': 15,
    'scheduler_patience': 7, 'scheduler_factor': 0.5,
}

# Extract features from NLOS train & val
nlos_train = train_df[train_df['Label'] == 1].reset_index(drop=True)
nlos_val = val_df[val_df['Label'] == 1].reset_index(drop=True)

feat_train_s2, _, _ = extract_features_from_df(nlos_train)
feat_val_s2, _, _ = extract_features_from_df(nlos_val)

# Auto-label: single-bounce (0) if Num_Peaks <= 2, multi-bounce (1) otherwise
threshold = PEAK_CONFIG['single_bounce_max_peaks']
y_train_s2 = (feat_train_s2['Num_Peaks'] > threshold).astype(float).values
y_val_s2 = (feat_val_s2['Num_Peaks'] > threshold).astype(float).values

print(f'  Train NLOS: {len(nlos_train)} — single={int((y_train_s2==0).sum())}, multi={int((y_train_s2==1).sum())}')
print(f'  Val NLOS:   {len(nlos_val)} — single={int((y_val_s2==0).sum())}, multi={int((y_val_s2==1).sum())}')

# Scale features
scaler_s2 = StandardScaler()
X_tr2 = torch.tensor(scaler_s2.fit_transform(feat_train_s2[FEATURE_NAMES].values.astype(np.float32))).to(device)
X_va2 = torch.tensor(scaler_s2.transform(feat_val_s2[FEATURE_NAMES].values.astype(np.float32))).to(device)
y_tr2 = torch.tensor(y_train_s2).unsqueeze(1).to(device)
y_va2 = torch.tensor(y_val_s2).unsqueeze(1).to(device)

# Class weight
n_single = int((y_train_s2 == 0).sum())
n_multi = int((y_train_s2 == 1).sum())
pos_weight_s2 = torch.tensor([n_single / max(n_multi, 1)], dtype=torch.float32).to(device)

train_ds2 = TensorDataset(X_tr2, y_tr2)
train_loader2 = DataLoader(train_ds2, batch_size=S2_CONFIG['batch_size'], shuffle=True)

model_s2 = BounceClassifier(
    input_size=len(FEATURE_NAMES),
    hidden_layers=S2_CONFIG['hidden_layers'],
    dropout=S2_CONFIG['dropout']
).to(device)

criterion_s2 = nn.BCEWithLogitsLoss(pos_weight=pos_weight_s2)
optimizer_s2 = optim.AdamW(model_s2.parameters(), lr=S2_CONFIG['lr'],
                           weight_decay=S2_CONFIG['weight_decay'])
scheduler_s2 = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer_s2, mode='max', factor=S2_CONFIG['scheduler_factor'],
    patience=S2_CONFIG['scheduler_patience'])

best_val_acc_s2 = 0
best_state_s2 = None
patience_s2 = 0

for epoch in range(S2_CONFIG['max_epochs']):
    model_s2.train()
    for bx, by in train_loader2:
        optimizer_s2.zero_grad()
        loss = criterion_s2(model_s2(bx), by)
        loss.backward()
        optimizer_s2.step()

    model_s2.eval()
    with torch.no_grad():
        vl = model_s2(X_va2)
        val_acc = ((torch.sigmoid(vl) > 0.5).float() == y_va2).float().mean().item()
    scheduler_s2.step(val_acc)

    if val_acc > best_val_acc_s2:
        best_val_acc_s2 = val_acc
        best_state_s2 = copy.deepcopy(model_s2.state_dict())
        patience_s2 = 0
    else:
        patience_s2 += 1

    if epoch % 10 == 0:
        print(f'  Ep {epoch:>3} | Val Acc: {100*val_acc:.2f}% | Best: {100*best_val_acc_s2:.2f}%')
    if patience_s2 >= S2_CONFIG['patience']:
        print(f'  Early stopping at epoch {epoch}')
        break

model_s2.load_state_dict(best_state_s2)
print(f'\nStage 2 Best Val Accuracy: {100*best_val_acc_s2:.2f}%')

STAGE 2: Training Bounce Classifier on NLOS subset of train set...
  Train NLOS: 1260 — single=584, multi=676
  Val NLOS:   270 — single=139, multi=131
  Ep   0 | Val Acc: 85.56% | Best: 85.56%
  Ep  10 | Val Acc: 94.44% | Best: 94.44%
  Ep  20 | Val Acc: 99.26% | Best: 99.26%
  Ep  30 | Val Acc: 99.63% | Best: 99.63%
  Ep  40 | Val Acc: 100.00% | Best: 100.00%
  Early stopping at epoch 48

Stage 2 Best Val Accuracy: 100.00%


In [7]:
# ==========================================
# STAGE 3 TRAINING: NLOS Bias Predictor (single-bounce NLOS only)
# ==========================================
print('STAGE 3: Training Bias Predictor on single-bounce NLOS...')
print('='*60)

S3_CONFIG = {
    'hidden_layers': [64, 32], 'dropout': 0.2,
    'batch_size': 32, 'max_epochs': 200, 'lr': 1e-3,
    'weight_decay': 1e-4, 'patience': 20,
    'scheduler_patience': 10, 'scheduler_factor': 0.5,
}

MEASURED_NLOS_BIAS = {'7.79m': 5.00, '10.77m': 5.32, '14m': 2.80}

# Filter single-bounce NLOS from train & val
def get_s3_data(feat_df, source_files):
    """Filter to single-bounce NLOS with known ground truth bias."""
    features, biases = [], []
    for i, fname in enumerate(source_files):
        grp = get_distance_group(fname)
        if grp not in MEASURED_NLOS_BIAS:
            continue
        if feat_df.iloc[i]['Num_Peaks'] > PEAK_CONFIG['single_bounce_max_peaks']:
            continue
        features.append(feat_df.iloc[i][FEATURE_NAMES].values)
        biases.append(MEASURED_NLOS_BIAS[grp])
    return np.array(features, dtype=np.float32), np.array(biases, dtype=np.float32)

feat_train_nlos, src_train_nlos, _ = extract_features_from_df(nlos_train)
feat_val_nlos, src_val_nlos, _ = extract_features_from_df(nlos_val)

X_train_s3_raw, y_train_s3 = get_s3_data(feat_train_nlos, src_train_nlos)
X_val_s3_raw, y_val_s3 = get_s3_data(feat_val_nlos, src_val_nlos)

print(f'  Train single-bounce NLOS: {len(X_train_s3_raw)}')
print(f'  Val single-bounce NLOS:   {len(X_val_s3_raw)}')

# Scale
scaler_s3 = StandardScaler()
X_tr3 = torch.tensor(scaler_s3.fit_transform(X_train_s3_raw)).to(device)
X_va3 = torch.tensor(scaler_s3.transform(X_val_s3_raw)).to(device)
y_tr3 = torch.tensor(y_train_s3).unsqueeze(1).to(device)
y_va3 = torch.tensor(y_val_s3).unsqueeze(1).to(device)

train_ds3 = TensorDataset(X_tr3, y_tr3)
train_loader3 = DataLoader(train_ds3, batch_size=S3_CONFIG['batch_size'], shuffle=True)

model_s3 = NLOSBiasPredictor(
    input_size=len(FEATURE_NAMES),
    hidden_layers=S3_CONFIG['hidden_layers'],
    dropout=S3_CONFIG['dropout']
).to(device)

criterion_s3 = nn.MSELoss()
optimizer_s3 = optim.AdamW(model_s3.parameters(), lr=S3_CONFIG['lr'],
                           weight_decay=S3_CONFIG['weight_decay'])
scheduler_s3 = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer_s3, mode='min', factor=S3_CONFIG['scheduler_factor'],
    patience=S3_CONFIG['scheduler_patience'])

best_val_mae_s3 = float('inf')
best_state_s3 = None
patience_s3 = 0

for epoch in range(S3_CONFIG['max_epochs']):
    model_s3.train()
    for bx, by in train_loader3:
        optimizer_s3.zero_grad()
        loss = criterion_s3(model_s3(bx), by)
        loss.backward()
        optimizer_s3.step()

    model_s3.eval()
    with torch.no_grad():
        vp3 = model_s3(X_va3)
        val_mae = torch.abs(vp3 - y_va3).mean().item()
    scheduler_s3.step(val_mae)

    if val_mae < best_val_mae_s3:
        best_val_mae_s3 = val_mae
        best_state_s3 = copy.deepcopy(model_s3.state_dict())
        patience_s3 = 0
    else:
        patience_s3 += 1

    if epoch % 20 == 0:
        print(f'  Ep {epoch:>3} | Val MAE: {val_mae:.4f}m | Best: {best_val_mae_s3:.4f}m')
    if patience_s3 >= S3_CONFIG['patience']:
        print(f'  Early stopping at epoch {epoch}')
        break

model_s3.load_state_dict(best_state_s3)
print(f'\nStage 3 Best Val MAE: {best_val_mae_s3:.4f}m')

STAGE 3: Training Bias Predictor on single-bounce NLOS...
  Train single-bounce NLOS: 584
  Val single-bounce NLOS:   139
  Ep   0 | Val MAE: 4.6260m | Best: 4.6260m
  Ep  20 | Val MAE: 0.3671m | Best: 0.3671m
  Ep  40 | Val MAE: 0.2310m | Best: 0.2310m
  Ep  60 | Val MAE: 0.1762m | Best: 0.1762m
  Ep  80 | Val MAE: 0.1806m | Best: 0.1693m
  Ep 100 | Val MAE: 0.1311m | Best: 0.1311m
  Ep 120 | Val MAE: 0.1348m | Best: 0.1197m
  Ep 140 | Val MAE: 0.1558m | Best: 0.1073m
  Early stopping at epoch 149

Stage 3 Best Val MAE: 0.1073m


---
## Section 5: End-to-End Pipeline Test on Unseen 15% Test Set

The test set has **never** been seen during training or validation.  
Run the full chained pipeline:
```
CIR → Stage 1 (LOS/NLOS?) → Stage 2 (bounce type?) → Stage 3 (bias) → d_corrected
```

In [8]:
# ==========================================
# END-TO-END PIPELINE TEST
# ==========================================
print('END-TO-END PIPELINE TEST ON UNSEEN TEST SET')
print('='*70)
print(f'Test set: {len(test_df)} samples (completely unseen)\n')

# --- Stage 1: LOS/NLOS Classification ---
X_test_s1, y_test_s1 = preprocess_stage1(test_df)
X_t1 = torch.tensor(X_test_s1).to(device)

model_s1.eval()
with torch.no_grad():
    s1_probs, _ = model_s1(X_t1)
    s1_preds = (s1_probs > 0.5).float().cpu().numpy().flatten()

s1_true = y_test_s1
s1_acc = (s1_preds == s1_true).mean()
print(f'STAGE 1 (LOS/NLOS Classification):')
print(f'  Accuracy: {100*s1_acc:.2f}%')
print(f'  Predicted LOS: {int((s1_preds==0).sum())}, NLOS: {int((s1_preds==1).sum())}')
print(f'  Actual    LOS: {int((s1_true==0).sum())}, NLOS: {int((s1_true==1).sum())}\n')

# --- Stage 2: Bounce Classification (on predicted NLOS) ---
# Get test samples that Stage 1 predicts as NLOS
nlos_mask_pred = s1_preds == 1
nlos_test_df = test_df[nlos_mask_pred].reset_index(drop=True)
nlos_test_actual_labels = s1_true[nlos_mask_pred]

feat_test_nlos, src_test_nlos, dist_test_nlos = extract_features_from_df(nlos_test_df)

# Ground truth bounce labels
y_test_bounce_true = (feat_test_nlos['Num_Peaks'] > PEAK_CONFIG['single_bounce_max_peaks']).astype(float).values

# Predict
X_t2 = torch.tensor(scaler_s2.transform(
    feat_test_nlos[FEATURE_NAMES].values.astype(np.float32)
)).to(device)

model_s2.eval()
with torch.no_grad():
    s2_logits = model_s2(X_t2)
    s2_preds = (torch.sigmoid(s2_logits) > 0.5).float().cpu().numpy().flatten()

s2_acc = (s2_preds == y_test_bounce_true).mean()
print(f'STAGE 2 (Bounce Classification — on Stage 1 NLOS predictions):')
print(f'  Samples entering Stage 2: {len(nlos_test_df)}')
print(f'  Accuracy: {100*s2_acc:.2f}%')
print(f'  Predicted single-bounce: {int((s2_preds==0).sum())}, multi: {int((s2_preds==1).sum())}')
print(f'  Actual    single-bounce: {int((y_test_bounce_true==0).sum())}, multi: {int((y_test_bounce_true==1).sum())}\n')

# --- Stage 3: Bias Prediction (on predicted single-bounce NLOS) ---
single_mask_pred = s2_preds == 0  # Stage 2 predicts single-bounce
single_test_df = nlos_test_df[single_mask_pred].reset_index(drop=True)
single_feat = feat_test_nlos[single_mask_pred].reset_index(drop=True)
single_src = [src_test_nlos[i] for i, m in enumerate(single_mask_pred) if m]
single_dist = [dist_test_nlos[i] for i, m in enumerate(single_mask_pred) if m]

# Get ground truth bias for these samples
results = []
X_s3_list = []
valid_indices = []

for i, fname in enumerate(single_src):
    grp = get_distance_group(fname)
    if grp not in MEASURED_NLOS_BIAS:
        continue
    gt = GROUND_TRUTH[grp]
    feat_vals = single_feat.iloc[i][FEATURE_NAMES].values.astype(np.float32)
    X_s3_list.append(feat_vals)
    valid_indices.append(i)
    results.append({
        'source_file': fname, 'group': grp,
        'd_uwb': single_dist[i],
        'd_direct': gt['d_direct'],
        'd_bounce': gt['d_bounce'],
        'actual_bias': gt['bias'],
    })

if len(X_s3_list) > 0:
    X_t3 = torch.tensor(scaler_s3.transform(
        np.array(X_s3_list)
    )).to(device)

    model_s3.eval()
    with torch.no_grad():
        s3_preds = model_s3(X_t3).cpu().numpy().flatten()

    for j, r in enumerate(results):
        r['ml_bias'] = s3_preds[j]
        r['d_corrected'] = r['d_uwb'] - s3_preds[j]
        r['bias_error'] = abs(s3_preds[j] - r['actual_bias'])
        r['correction_error'] = abs(r['d_corrected'] - r['d_direct'])

results_df = pd.DataFrame(results)
test_mae = results_df['bias_error'].mean()
test_rmse = np.sqrt((results_df['bias_error']**2).mean())

print(f'STAGE 3 (Bias Prediction — on predicted single-bounce NLOS):')
print(f'  Samples entering Stage 3: {len(results_df)}')
print(f'  Bias MAE:  {test_mae:.4f}m')
print(f'  Bias RMSE: {test_rmse:.4f}m')
print(f'  Correction MAE: {results_df["correction_error"].mean():.4f}m')

print(f'\n{"="*70}')
print(f'END-TO-END PIPELINE SUMMARY (on unseen test set)')
print(f'{"="*70}')
print(f'  Stage 1 (LOS/NLOS):        {100*s1_acc:.2f}% accuracy')
print(f'  Stage 2 (bounce type):     {100*s2_acc:.2f}% accuracy')
print(f'  Stage 3 (bias prediction): MAE = {test_mae:.4f}m')
print(f'  Distance correction:       MAE = {results_df["correction_error"].mean():.4f}m')
print(f'\n  Pipeline flow: {len(test_df)} total → {int(nlos_mask_pred.sum())} NLOS → {int(single_mask_pred.sum())} single-bounce → {len(results_df)} with GT')

END-TO-END PIPELINE TEST ON UNSEEN TEST SET
Test set: 540 samples (completely unseen)

STAGE 1 (LOS/NLOS Classification):
  Accuracy: 99.63%
  Predicted LOS: 272, NLOS: 268
  Actual    LOS: 270, NLOS: 270

STAGE 2 (Bounce Classification — on Stage 1 NLOS predictions):
  Samples entering Stage 2: 268
  Accuracy: 98.51%
  Predicted single-bounce: 126, multi: 142
  Actual    single-bounce: 122, multi: 146

STAGE 3 (Bias Prediction — on predicted single-bounce NLOS):
  Samples entering Stage 3: 126
  Bias MAE:  0.1781m
  Bias RMSE: 0.4719m
  Correction MAE: 4.0628m

END-TO-END PIPELINE SUMMARY (on unseen test set)
  Stage 1 (LOS/NLOS):        99.63% accuracy
  Stage 2 (bounce type):     98.51% accuracy
  Stage 3 (bias prediction): MAE = 0.1781m
  Distance correction:       MAE = 4.0628m

  Pipeline flow: 540 total → 268 NLOS → 126 single-bounce → 126 with GT


---
## Section 6: Test Set Visualization

Comprehensive plots showing pipeline performance on the unseen test set.

In [ ]:
# ==========================================
# VISUALIZATION: 4-PANEL RESULTS
# ==========================================
fig, axs = plt.subplots(2, 2, figsize=(20, 16))
plt.subplots_adjust(hspace=0.35, wspace=0.3)

# --- Panel 1: Stage 1 Confusion Matrix ---
ax = axs[0, 0]
cm1 = confusion_matrix(s1_true, s1_preds)
disp1 = ConfusionMatrixDisplay(cm1, display_labels=['LOS', 'NLOS'])
disp1.plot(ax=ax, cmap='Blues', colorbar=False)
ax.set_title(f'Stage 1: LOS/NLOS Classification\nAccuracy: {100*s1_acc:.2f}%',
             fontsize=14, fontweight='bold')

# --- Panel 2: Stage 2 Confusion Matrix ---
ax = axs[0, 1]
cm2 = confusion_matrix(y_test_bounce_true, s2_preds)
disp2 = ConfusionMatrixDisplay(cm2, display_labels=['Single', 'Multi'])
disp2.plot(ax=ax, cmap='Greens', colorbar=False)
ax.set_title(f'Stage 2: Bounce Classification\nAccuracy: {100*s2_acc:.2f}%',
             fontsize=14, fontweight='bold')

# --- Panel 3: Stage 3 Actual vs Predicted Bias ---
ax = axs[1, 0]
colors_map = {'7.79m': '#e74c3c', '10.77m': '#3498db', '14m': '#27ae60'}
for grp in sorted(GROUND_TRUTH.keys()):
    mask = results_df['group'] == grp
    sub = results_df[mask]
    if len(sub) > 0:
        ax.scatter(sub['actual_bias'], sub['ml_bias'], alpha=0.5, s=30,
                   color=colors_map[grp], label=f'{grp} (n={len(sub)})',
                   edgecolors='white', linewidth=0.5)

all_bias = np.concatenate([results_df['actual_bias'].values, results_df['ml_bias'].values])
lims = [all_bias.min() - 0.5, all_bias.max() + 0.5]
ax.plot(lims, lims, 'k--', lw=2, alpha=0.7, label='Perfect prediction')
ax.set_xlabel('Actual NLOS Bias (m)', fontsize=12)
ax.set_ylabel('Predicted NLOS Bias (m)', fontsize=12)
ax.set_title(f'Stage 3: Bias Prediction (Test Set)\nMAE = {test_mae:.3f}m',
             fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# --- Panel 4: Distance Correction ---
ax = axs[1, 1]
groups_sorted = sorted(GROUND_TRUTH.keys())
x_pos = np.arange(len(groups_sorted))
width = 0.25

d_uwb_vals, d_corr_vals, d_direct_vals = [], [], []
for grp in groups_sorted:
    gt = GROUND_TRUTH[grp]
    sub = results_df[results_df['group'] == grp]
    d_uwb_vals.append(sub['d_uwb'].mean() if len(sub) > 0 else 0)
    d_corr_vals.append(sub['d_corrected'].mean() if len(sub) > 0 else 0)
    d_direct_vals.append(gt['d_direct'])

ax.bar(x_pos - width, d_uwb_vals, width, label='d_UWB (biased)',
       color='#e74c3c', alpha=0.8, edgecolor='black')
ax.bar(x_pos, d_corr_vals, width, label='d_corrected (ML)',
       color='#27ae60', alpha=0.8, edgecolor='black')
ax.bar(x_pos + width, d_direct_vals, width, label='d_direct (truth)',
       color='#333333', alpha=0.8, edgecolor='black')

for i, (u, c, d) in enumerate(zip(d_uwb_vals, d_corr_vals, d_direct_vals)):
    if u > 0:
        ax.text(i - width, u + 0.15, f'{u:.1f}', ha='center', fontsize=10, fontweight='bold', color='#c0392b')
        ax.text(i, c + 0.15, f'{c:.1f}', ha='center', fontsize=10, fontweight='bold', color='#1e8449')
        ax.text(i + width, d + 0.15, f'{d:.1f}', ha='center', fontsize=10, fontweight='bold')

ax.set_xticks(x_pos)
ax.set_xticklabels(groups_sorted, fontsize=12)
ax.set_ylabel('Distance (m)', fontsize=12)
ax.set_title(f'Distance Correction: d_corrected = d_UWB - bias\nCorrection MAE = {results_df["correction_error"].mean():.3f}m',
             fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, axis='y')

fig.suptitle(
    f'End-to-End Pipeline Results — Unseen Test Set ({len(test_df)} samples)\n'
    f'70% Train / 15% Val / 15% Test Split',
    fontsize=16, fontweight='bold', y=1.02
)
plt.tight_layout()
plt.show()

In [ ]:
# ==========================================
# BOUNCE DISTANCE BENCHMARK vs Xu Xueli (2024)
# ==========================================
# Derive d_bounce from bias predictions
results_df['actual_d_bounce'] = results_df['group'].map(
    {g: v['d_bounce'] for g, v in GROUND_TRUTH.items()}
)
results_df['predicted_d_bounce'] = results_df.apply(
    lambda r: GROUND_TRUTH[r['group']]['d_direct'] + r['ml_bias'], axis=1
)

our_db_mae = mean_absolute_error(results_df['actual_d_bounce'], results_df['predicted_d_bounce'])
our_db_rmse = np.sqrt(mean_squared_error(results_df['actual_d_bounce'], results_df['predicted_d_bounce']))
our_db_r2 = r2_score(results_df['actual_d_bounce'], results_df['predicted_d_bounce'])

# Xu Xueli's reported metrics
xu_mae, xu_rmse, xu_r2 = 0.34594, 0.60008, 0.61868

fig, axs = plt.subplots(1, 2, figsize=(20, 8))

# Scatter plot
ax = axs[0]
for grp in sorted(GROUND_TRUTH.keys()):
    sub = results_df[results_df['group'] == grp]
    if len(sub) > 0:
        ax.scatter(sub['actual_d_bounce'], sub['predicted_d_bounce'],
                   alpha=0.5, s=40, color=colors_map[grp],
                   edgecolors='white', linewidth=0.5,
                   label=f'{grp} (n={len(sub)})')

all_db = np.concatenate([results_df['actual_d_bounce'].values, results_df['predicted_d_bounce'].values])
lims = [all_db.min() - 1, all_db.max() + 1]
ax.plot(lims, lims, 'k--', lw=2, alpha=0.7)
ax.set_xlabel('Actual 1-Bounce Distance (m)', fontsize=13)
ax.set_ylabel('Predicted 1-Bounce Distance (m)', fontsize=13)
ax.set_title(f'Ours — Unseen Test Set\nMAE={our_db_mae:.3f}m  R²={our_db_r2:.3f}', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xlim(lims); ax.set_ylim(lims)
ax.set_aspect('equal')

# Metrics comparison
ax = axs[1]
labels = ['MAE (m)', 'RMSE (m)', 'R²']
xu_v = [xu_mae, xu_rmse, xu_r2]
our_v = [our_db_mae, our_db_rmse, our_db_r2]
x = np.arange(len(labels))
w = 0.35
ax.bar(x - w/2, xu_v, w, label='Xu Xueli (2024)\nEncoder + RF', color='#e74c3c', alpha=0.8, edgecolor='black')
ax.bar(x + w/2, our_v, w, label='Ours (Test Set)\nMLP + CIR Features', color='#27ae60', alpha=0.8, edgecolor='black')
for bar, val in zip(ax.containers[0], xu_v):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
            f'{val:.3f}', ha='center', fontsize=12, fontweight='bold', color='#c0392b')
for bar, val in zip(ax.containers[1], our_v):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
            f'{val:.3f}', ha='center', fontsize=12, fontweight='bold', color='#1e8449')
ax.set_xticks(x)
ax.set_xticklabels(labels, fontsize=12)
ax.set_title('Bounce Distance: Ours vs Xu Xueli (2024)', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')

fig.suptitle('1-Bounce Distance Prediction — Unseen Test Set vs Prior Work',
             fontsize=15, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f'\nBounce Distance Benchmark (Unseen Test Set):')
print(f'  {"Metric":<8} | {"Xu Xueli":<12} | {"Ours":<12} | Improvement')
print(f'  {"-"*48}')
print(f'  {"MAE":<8} | {xu_mae:<11.3f}m | {our_db_mae:<11.3f}m | {xu_mae/our_db_mae:.1f}x lower')
print(f'  {"RMSE":<8} | {xu_rmse:<11.3f}m | {our_db_rmse:<11.3f}m | {xu_rmse/our_db_rmse:.1f}x lower')
print(f'  {"R²":<8} | {xu_r2:<12.3f} | {our_db_r2:<12.3f} | +{our_db_r2 - xu_r2:.3f}')

---
## Section 7: Save Production Models

Save the models trained on the 70% train split for deployment.

In [11]:
# ==========================================
# SAVE PRODUCTION MODELS
# ==========================================
torch.save(model_s1.state_dict(), 'prod_stage1_pi_hlnn.pt')
torch.save(model_s2.state_dict(), 'prod_stage2_bounce.pt')
torch.save(model_s3.state_dict(), 'prod_stage3_bias.pt')

torch.save({
    'stage1_config': S1_CONFIG,
    'stage2_config': S2_CONFIG,
    'stage3_config': S3_CONFIG,
    'scaler_s2_mean': scaler_s2.mean_,
    'scaler_s2_scale': scaler_s2.scale_,
    'scaler_s3_mean': scaler_s3.mean_,
    'scaler_s3_scale': scaler_s3.scale_,
    'feature_names': FEATURE_NAMES,
    'ground_truth': GROUND_TRUTH,
    'measured_nlos_bias': MEASURED_NLOS_BIAS,
    'peak_config': PEAK_CONFIG,
}, 'prod_pipeline_config.pt')

print('Saved production models:')
print('  prod_stage1_pi_hlnn.pt')
print('  prod_stage2_bounce.pt')
print('  prod_stage3_bias.pt')
print('  prod_pipeline_config.pt')

Saved production models:
  prod_stage1_pi_hlnn.pt
  prod_stage2_bounce.pt
  prod_stage3_bias.pt
  prod_pipeline_config.pt


In [12]:
# ==========================================
# FINAL SUMMARY
# ==========================================
print('='*70)
print('MULTI-MODEL PIPELINE — FINAL RESULTS')
print('='*70)
print(f'\nData Split: 70/15/15 (Train={len(train_df)}, Val={len(val_df)}, Test={len(test_df)})')
print(f'\nPipeline Architecture:')
print(f'  Stage 1: PI-HLNN (Liquid Neural Network)  — {sum(p.numel() for p in model_s1.parameters()):,} params')
print(f'  Stage 2: MLP Bounce Classifier             — {sum(p.numel() for p in model_s2.parameters()):,} params')
print(f'  Stage 3: MLP NLOS Bias Predictor            — {sum(p.numel() for p in model_s3.parameters()):,} params')
total_params = sum(p.numel() for p in model_s1.parameters()) + \
               sum(p.numel() for p in model_s2.parameters()) + \
               sum(p.numel() for p in model_s3.parameters())
print(f'  Total pipeline:                             — {total_params:,} params')
print(f'\nResults on Unseen Test Set ({len(test_df)} samples):')
print(f'  Stage 1 (LOS/NLOS):      {100*s1_acc:.2f}% accuracy')
print(f'  Stage 2 (Bounce type):   {100*s2_acc:.2f}% accuracy')
print(f'  Stage 3 (Bias):          MAE = {test_mae:.4f}m')
print(f'  Distance correction:     MAE = {results_df["correction_error"].mean():.4f}m')
print(f'\nBenchmark vs Xu Xueli (2024):')
print(f'  Bounce distance MAE: {our_db_mae:.3f}m (ours) vs {xu_mae:.3f}m (Xu Xueli)')
print(f'  Bounce distance R²:  {our_db_r2:.3f} (ours) vs {xu_r2:.3f} (Xu Xueli)')
print(f'\nPipeline flow on test set:')
print(f'  {len(test_df)} total')
print(f'  → {int(nlos_mask_pred.sum())} classified as NLOS (Stage 1)')
print(f'  → {int(single_mask_pred.sum())} classified as single-bounce (Stage 2)')
print(f'  → {len(results_df)} bias predicted + distance corrected (Stage 3)')
print(f'\nAt inference: d_corrected = d_UWB - predicted_bias')

MULTI-MODEL PIPELINE — FINAL RESULTS

Data Split: 70/15/15 (Train=2520, Val=540, Test=540)

Pipeline Architecture:
  Stage 1: PI-HLNN (Liquid Neural Network)  — 10,625 params
  Stage 2: MLP Bounce Classifier             — 769 params
  Stage 3: MLP NLOS Bias Predictor            — 2,561 params
  Total pipeline:                             — 13,955 params

Results on Unseen Test Set (540 samples):
  Stage 1 (LOS/NLOS):      99.63% accuracy
  Stage 2 (Bounce type):   98.51% accuracy
  Stage 3 (Bias):          MAE = 0.1781m
  Distance correction:     MAE = 4.0628m

Benchmark vs Xu Xueli (2024):
  Bounce distance MAE: 0.178m (ours) vs 0.346m (Xu Xueli)
  Bounce distance R²:  0.918 (ours) vs 0.619 (Xu Xueli)

Pipeline flow on test set:
  540 total
  → 268 classified as NLOS (Stage 1)
  → 126 classified as single-bounce (Stage 2)
  → 126 bias predicted + distance corrected (Stage 3)

At inference: d_corrected = d_UWB - predicted_bias
